In [1]:
import pandas as pd
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer

In [2]:
load_dotenv()

True

In [3]:
import os

In [4]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [32]:
df = pd.read_csv("./tmdb_5000_movies.csv", encoding="utf-8")

In [38]:

df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [39]:
import json

def safe_parse(val):
    if pd.isna(val) or val == "" or val == "[]":
        return []
    try:
        return json.loads(val.replace("'", '"'))  # đổi ' thành " để JSON đọc được
    except Exception:
        try:
            return ast.literal_eval(val)  # fallback nếu JSON lỗi
        except Exception:
            return []

def build_desc(row):
    genres = [g.get("name", "") for g in safe_parse(row["genres"])]
    keywords = [k.get("name", "") for k in safe_parse(row["keywords"])]

    genres_str = ", ".join([g for g in genres if g])
    keywords_str = ", ".join([k for k in keywords if k])

    return f"""{row['title']} Genres: {genres_str} Keywords: {keywords_str} Overview: {row['overview']} Tagline: {row['tagline']}"""


In [40]:
df["desc"] = df.apply(build_desc, axis=1)

In [41]:
print(df["desc"][0])

Avatar Genres: Action, Adventure, Fantasy, Science Fiction Keywords: culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d Overview: In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Tagline: Enter the World of Pandora.


In [42]:
print(df.loc[df["title"]=="Sherlock Holmes: A Game of Shadows", "desc"].values[0])

Sherlock Holmes: A Game of Shadows Genres: Adventure, Action, Crime, Mystery Keywords: detective inspector, steampunk, criminal mastermind Overview: There is a new criminal mastermind at large (Professor Moriarty) and not only is he Holmes’ intellectual equal, but his capacity for evil and lack of conscience may give him an advantage over the  detective. Tagline: The game is afoot.


In [6]:
from pinecone import Pinecone
import os


In [7]:
from transformers import pipeline

In [7]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [37]:
def embed_text(text):
    return model.encode(text).tolist()

In [38]:
df["embedding"] = df["desc"].apply(embed_text)

In [39]:
print(df[["title", "embedding"]].head(2))
print("Chiều vector:", len(df["embedding"].iloc[0]))

                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   

                                           embedding  
0  [0.00427649961784482, -0.3326219320297241, 0.3...  
1  [0.09962689876556396, -0.12092940509319305, 0....  
Chiều vector: 384


# PineCOne connect

In [8]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("movies")

In [47]:
import math

def clean_metadata(val):
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return ""  # thay NaN bằng chuỗi rỗng
    return str(val)

vectors = []
for _, row in df.iterrows():
    vectors.append({
        "id": str(row["id"]),
        "values": row["embedding"],
        "metadata": {
            "title": clean_metadata(row["title"]),
            "overview": clean_metadata(row["overview"]),
            "tagline": clean_metadata(row["tagline"])
        }
    })

In [49]:
def upsert_in_batches(index, vectors, batch_size=100):
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i: i + batch_size]
        index.upsert(batch)
        print(f"✅ Upsert batch {i//batch_size + 1} ({len(batch)} vectors)")


In [50]:
upsert_in_batches(index, vectors, batch_size=100)

✅ Upsert batch 1 (100 vectors)
✅ Upsert batch 2 (100 vectors)
✅ Upsert batch 3 (100 vectors)
✅ Upsert batch 4 (100 vectors)
✅ Upsert batch 5 (100 vectors)
✅ Upsert batch 6 (100 vectors)
✅ Upsert batch 7 (100 vectors)
✅ Upsert batch 8 (100 vectors)
✅ Upsert batch 9 (100 vectors)
✅ Upsert batch 10 (100 vectors)
✅ Upsert batch 11 (100 vectors)
✅ Upsert batch 12 (100 vectors)
✅ Upsert batch 13 (100 vectors)
✅ Upsert batch 14 (100 vectors)
✅ Upsert batch 15 (100 vectors)
✅ Upsert batch 16 (100 vectors)
✅ Upsert batch 17 (100 vectors)
✅ Upsert batch 18 (100 vectors)
✅ Upsert batch 19 (100 vectors)
✅ Upsert batch 20 (100 vectors)
✅ Upsert batch 21 (100 vectors)
✅ Upsert batch 22 (100 vectors)
✅ Upsert batch 23 (100 vectors)
✅ Upsert batch 24 (100 vectors)
✅ Upsert batch 25 (100 vectors)
✅ Upsert batch 26 (100 vectors)
✅ Upsert batch 27 (100 vectors)
✅ Upsert batch 28 (100 vectors)
✅ Upsert batch 29 (100 vectors)
✅ Upsert batch 30 (100 vectors)
✅ Upsert batch 31 (100 vectors)
✅ Upsert batch 32

In [ ]:
print(f"✅ Upsert {len(vectors)} movies vào Pinecone thành công!")

In [24]:
translator_vi2en = pipeline("translation", model="Helsinki-NLP/opus-mt-vi-en")

config.json: 0.00B [00:00, ?B/s]

D:\ai-env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chu Dung\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-vi-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For bette

pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [19]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-vi")

config.json: 0.00B [00:00, ?B/s]

D:\ai-env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chu Dung\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-vi. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For bette

pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

D:\ai-env\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [9]:
def find_similar_movies(vietnamese_query, top_k=5):
    """
    Hàm này nhận một câu mô tả tiếng Việt, embed nó,
    và truy vấn Pinecone để tìm các phim phù hợp.
    """
    print(f"Đang tìm kiếm cho: '{vietnamese_query}'\n")

    # 1. Embed câu truy vấn tiếng Việt bằng CÙNG một model đa ngôn ngữ
    query_embedding = model.encode(vietnamese_query).tolist()

    # 2. Truy vấn Pinecone
    try:
        query_results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True  # Lấy cả metadata (title, overview)
        )

        # 3. In kết quả
        print("Kết quả đề xuất:")
        if not query_results['matches']:
            print("Không tìm thấy kết quả phù hợp.")
            return

        for result in query_results['matches']:
            print(f"  - ID Phim: {result['id']}")
            print(f"    Tiêu đề: {result['metadata']['title']}")
            print(f"    Tóm tắt: {result['metadata']['overview']}")
            print(f"    Tagline: {result['metadata']['tagline']}")
            print(f"    Độ tương đồng (Score): {result['score']:.4f}\n")

        return query_results

    except Exception as e:
        print(f"Lỗi khi truy vấn Pinecone: {e}")
        return None

# --- Ví dụ sử dụng ---
print("="*30)
print("BẮT ĐẦU TRUY VẤN")
print("="*30)

# Ví dụ 1: Phim về không gian, người ngoài hành tinh
vietnamese_query_1 = "một bộ phim về người ngoài hành tinh xâm chiếm trái đất và văn hóa xung đột"
find_similar_movies(vietnamese_query_1)

print("-" * 20)

# Ví dụ 2: Phim tình cảm lãng mạn
vietnamese_query_2 = "câu chuyện tình yêu lãng mạn trên một con tàu lớn bị chìm"
find_similar_movies(vietnamese_query_2)

print("-" * 20)

# Ví dụ 3: Phim hành động
vietnamese_query_3 = "người dơi chiến đấu với một tên hề tội phạm"
find_similar_movies(vietnamese_query_3)

BẮT ĐẦU TRUY VẤN
Đang tìm kiếm cho: 'một bộ phim về người ngoài hành tinh xâm chiếm trái đất và văn hóa xung đột'

Kết quả đề xuất:
  - ID Phim: 4858
    Tiêu đề: The Invasion
    Tóm tắt: Washington, D.C. psychologist Carol Bennell and her colleague Dr. Ben Driscoll are the only two people on Earth who are aware of an epidemic running rampant through the city. They discover an alien virus aboard a crashed space shuttle that transforms anyone who comes into contact with it into unfeeling drones while they sleep. Carol realizes her son holds the key to stopping the spread of the plague and she races to find him before it is too late.
    Tagline: Do not trust anyone. Do not show emotion. Do not fall asleep.
    Độ tương đồng (Score): 0.6638

  - ID Phim: 34086
    Tiêu đề: Spaced Invaders
    Tóm tắt: A crew of Martians overhears a radio broadcast of Orson Welles' "War of the Worlds" coming from Earth, and, thinking the Martian fleet is attacking Earth, they land their broken-down ship 

{'matches': [{'id': '13056',
              'metadata': {'overview': 'Waging his one-man war on the world of '
                                       'organized crime, ruthless '
                                       'vigilante-hero Frank Castle sets his '
                                       'sights on overeager mob boss Billy '
                                       'Russoti. After Russoti is left '
                                       'horribly disfigured by Castle, he sets '
                                       'out for vengeance under his new alias: '
                                       'Jigsaw. With the "Punisher Task Force" '
                                       'hot on his trail and the FBI unable to '
                                       'take Jigsaw in, Frank must stand up to '
                                       'the formidable army that Jigsaw has '
                                       'recruited before more of his evil '
                                  

# Model mới

In [12]:
print("Đang tải model 'paraphrase-multilingual-mpnet-base-v2'...")
print("(Việc này có thể mất vài phút nếu đây là lần đầu tiên)")
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
print("✅ Tải model thành công.")


Đang tải model 'paraphrase-multilingual-mpnet-base-v2'...
(Việc này có thể mất vài phút nếu đây là lần đầu tiên)
✅ Tải model thành công.


In [34]:
try:
    df = pd.read_csv("./tmdb_5000_movies.csv", encoding="utf-8")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file 'tmdb_5000_movies.csv'.")
    exit()

In [35]:
df.iloc[0]['genres']


'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [36]:
import math

In [37]:
def clean_metadata(val):
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return ""  # thay NaN bằng chuỗi rỗng
    return str(val)

In [38]:
import pandas as pd
import json

# Hàm helper để parse chuỗi JSON một cách an toàn
def safe_parse(text_data):
    # Trả về list rỗng nếu dữ liệu là NaN, rỗng hoặc "[]"
    if pd.isna(text_data) or text_data == "" or text_data == "[]":
        return []
    try:
        # Thay thế dấu nháy đơn ' bằng nháy kép " để biến nó thành JSON hợp lệ
        # và dùng json.loads để đọc.
        # Ví dụ: '[{"id": 28, "name": "Action"}]' -> '[{"id": 28, "name": "Action"}]'
        return json.loads(text_data)
    except Exception as e:
        # Nếu có lỗi, in ra để debug và trả về list rỗng
        print(f"Lỗi khi parse: {e} với dữ liệu: {text_data}")
        return []

# Hàm chính để xây dựng mô tả phim
def build_description(row):
    """
    Hàm này nhận vào một hàng (row) của DataFrame và tạo ra một chuỗi
    mô tả tổng hợp cho bộ phim đó.
    """
    # Lấy ra danh sách tên 'name' từ cột 'genres'
    # Lưu ý: Tên cột trong file csv là "genres" (có chữ s)
    genres_list = [item.get("name", "") for item in safe_parse(row["genres"])]
    
    # Lấy ra danh sách tên 'name' từ cột 'keywords'
    keywords_list = [item.get("name", "") for item in safe_parse(row["keywords"])]

    # Nối các tên lại thành một chuỗi, cách nhau bởi dấu phẩy
    # Ví dụ: ["Action", "Adventure"] -> "Action, Adventure"
    genres_str = ", ".join([g for g in genres_list if g])
    keywords_str = ", ".join([k for k in keywords_list if k])

    # Dùng f-string để tạo mô tả cuối cùng
    # Chúng ta thêm các nhãn như "Genres:", "Keywords:" để model hiểu rõ ngữ cảnh
    description = f"""Title: {row['title']}.
Genres: {genres_str}.
Keywords: {keywords_str}.
Overview: {row['overview']}.
Tagline: {row['tagline']}"""
    
    return description


In [39]:
df["desc"] = df.apply(build_description, axis=1)
print(df["desc"][0])

Title: Avatar.
Genres: Action, Adventure, Fantasy, Science Fiction.
Keywords: culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d.
Overview: In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization..
Tagline: Enter the World of Pandora.


In [40]:
print("Đang xử lý và tạo mô tả cho từng bộ phim...")
df["desc"] = df.apply(build_description, axis=1)
print("✔️ Xử lý mô tả phim hoàn tất.")

Đang xử lý và tạo mô tả cho từng bộ phim...
✔️ Xử lý mô tả phim hoàn tất.


In [41]:
df['desc'][4802]

"Title: My Date with Drew.\nGenres: Documentary.\nKeywords: obsession, camcorder, crush, dream girl.\nOverview: Ever since the second grade when he first saw her in E.T. The Extraterrestrial, Brian Herzlinger has had a crush on Drew Barrymore. Now, 20 years later he's decided to try to fulfill his lifelong dream by asking her for a date. There's one small problem: She's Drew Barrymore and he's, well, Brian Herzlinger, a broke 27-year-old aspiring filmmaker from New Jersey..\nTagline: nan"

In [48]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "movies"
index = pc.Index(index_name)
print(f"Đã kết nối tới Pinecone index '{index_name}'.")

Đã kết nối tới Pinecone index 'movies'.


In [43]:
print(f"Đang xóa tất cả các vector cũ trong index '{index_name}'...")
try:
    index.delete(deleteAll=True)
    print("✔️ Xóa index cũ thành công.")
except Exception as e:
    print(f"Thông báo: Không thể xóa index (có thể index đã trống). Lỗi: {e}")


Đang xóa tất cả các vector cũ trong index 'movies'...
Thông báo: Không thể xóa index (có thể index đã trống). Lỗi: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 14 Oct 2025 03:25:56 GMT', 'Content-Type': 'application/json', 'Content-Length': '55', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '20', 'x-pinecone-request-id': '4431379502526659453', 'x-envoy-upstream-service-time': '21', 'server': 'envoy'})
HTTP response body: {"code":5,"message":"Namespace not found","details":[]}



In [44]:
# --- 6. TẠO EMBEDDING MỚI VÀ UPSERT ---
print("Bắt đầu tạo embedding cho tất cả phim với model mới...")
df["embedding"] = df["desc"].apply(lambda text: model.encode(text).tolist())
print("✔️ Tạo embedding mới thành công.")

Bắt đầu tạo embedding cho tất cả phim với model mới...
✔️ Tạo embedding mới thành công.


In [45]:
# ĐỊNH NGHĨA HÀM XỬ LÝ NaN (ĐÚNG NHƯ BẠN YÊU CẦU)
def clean_metadata(val):
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return ""  # thay NaN hoặc None bằng chuỗi rỗng
    return str(val)

# ÁP DỤNG HÀM XỬ LÝ NaN TẠI ĐÂY
print("Chuẩn bị vector và metadata để upsert...")
vectors = [
    {
        "id": str(row["id"]),
        "values": row["embedding"],
        "metadata": {
            "title": clean_metadata(row["title"]),
            "overview": clean_metadata(row["overview"]),
            "tagline": clean_metadata(row["tagline"])
        }
    }
    for _, row in df.iterrows()
]
print("✔️ Chuẩn bị vector thành công.")

Chuẩn bị vector và metadata để upsert...
✔️ Chuẩn bị vector thành công.


In [50]:
def upsert_in_batches(index_obj, vectors_list, batch_size=100):
    print(f"Bắt đầu upsert {len(vectors_list)} vector lên Pinecone...")
    for i in range(0, len(vectors_list), batch_size):
        batch = vectors_list[i: i + batch_size]
        index_obj.upsert(batch)
        print(f"  -> Đã upsert batch {i//batch_size + 1}/{len(vectors_list)//batch_size + 1}")

upsert_in_batches(index, vectors)
print(f"🚀 HOÀN THÀNH! Đã re-index thành công {len(vectors)} phim vào Pinecone!")


Bắt đầu upsert 4803 vector lên Pinecone...
  -> Đã upsert batch 1/49
  -> Đã upsert batch 2/49
  -> Đã upsert batch 3/49
  -> Đã upsert batch 4/49
  -> Đã upsert batch 5/49
  -> Đã upsert batch 6/49
  -> Đã upsert batch 7/49
  -> Đã upsert batch 8/49
  -> Đã upsert batch 9/49
  -> Đã upsert batch 10/49
  -> Đã upsert batch 11/49
  -> Đã upsert batch 12/49
  -> Đã upsert batch 13/49
  -> Đã upsert batch 14/49
  -> Đã upsert batch 15/49
  -> Đã upsert batch 16/49
  -> Đã upsert batch 17/49
  -> Đã upsert batch 18/49
  -> Đã upsert batch 19/49
  -> Đã upsert batch 20/49
  -> Đã upsert batch 21/49
  -> Đã upsert batch 22/49
  -> Đã upsert batch 23/49
  -> Đã upsert batch 24/49
  -> Đã upsert batch 25/49
  -> Đã upsert batch 26/49
  -> Đã upsert batch 27/49
  -> Đã upsert batch 28/49
  -> Đã upsert batch 29/49
  -> Đã upsert batch 30/49
  -> Đã upsert batch 31/49
  -> Đã upsert batch 32/49
  -> Đã upsert batch 33/49
  -> Đã upsert batch 34/49
  -> Đã upsert batch 35/49
  -> Đã upsert batch 

In [9]:
TRANSLATION_MODEL = "Helsinki-NLP/opus-mt-vi-en"

translator = pipeline("translation", model=TRANSLATION_MODEL)

D:\ai-env\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [10]:
def find_similar_movies_final(vietnamese_query, top_k=5):
    """
    Hàm tìm kiếm phiên bản cuối cùng:
    1. Dịch câu truy vấn Tiếng Việt sang Tiếng Anh.
    2. Dùng câu Tiếng Anh để tạo vector và tìm kiếm.
    """
    print("\n" + "="*50)
    print(f"🎬 Bắt đầu tìm kiếm cho query (VI): '{vietnamese_query}'")
    
    # BƯỚC 1: DỊCH SANG TIẾNG ANH
    translated_result = translator(vietnamese_query)
    english_query = translated_result[0]['translation_text']
    print(f"   -> Đã dịch sang (EN): '{english_query}'")
    
    # BƯỚC 2: TẠO VECTOR TỪ CÂU TIẾNG ANH
    query_embedding = model.encode(english_query).tolist()
    
    # BƯỚC 3: TRUY VẤN PINECONE
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    # BƯỚC 4: IN KẾT QUẢ
    print("\n✨ Kết quả đề xuất: ✨")
    if not results['matches']:
        print("   Không tìm thấy phim phù hợp.")
        return

    for match in results['matches']:
        title = match['metadata'].get('title', 'N/A')
        overview = match['metadata'].get('overview', 'No overview available.')
        score = match['score']
        print(f"  - Tiêu đề: {title} (Score: {score:.2f})")
        # In thêm overview để dễ kiểm tra
        # print(f"    Tóm tắt: {overview[:150]}...") 
    print("="*50)

In [13]:
find_similar_movies_final("Người dơi đánh nhau với tên cướp")


🎬 Bắt đầu tìm kiếm cho query (VI): 'Người dơi đánh nhau với tên cướp'
   -> Đã dịch sang (EN): 'Bats fight thieves.'

✨ Kết quả đề xuất: ✨
  - Tiêu đề: Bats (Score: 0.61)
  - Tiêu đề: Robin Hood: Prince of Thieves (Score: 0.41)
  - Tiêu đề: Batman & Robin (Score: 0.41)
  - Tiêu đề: Snatch (Score: 0.41)
  - Tiêu đề: The Benchwarmers (Score: 0.40)
